We test and learn the usage of the nloptr wrapper of python. The examples have been prepared by referring to https://nlopt.readthedocs.io/en/latest/NLopt_Python_Reference/

In [74]:
#!pip install nlopt
import numpy as np
import nlopt
from numpy import *

NLOpt addrsses non-linear optimization problems of the form:


$min f(x)$

st  $g(x) <=0$   , $h(x) = 0$,  $lb <= x <= ub$



First we create nloptr object using its constructor. This contstructor needs **algorithm** to be specified and the dimentionality of the problem **n**

https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/

Algorithms are specified by numerical constants such as nlopt.LD_SLSQP,nlopt.LN_COBYLA etc..

Objective function is set by calling the **set_min_objective** or **set_max_objective**. Let as try to minimze $z = (x-2)^2  + (y - 3)^2$. Clearly this will have a minima at $(2,3)$

THe objective funtion's first parameter will be the numpy array of length n.

The second parameter is gradient array of length n, where $i^{th}$ elemtn is set inside the function with derivatiove $\frac{\partial f}{\partial x_i}$

Bounds constraints are set by **set_lower_bounds** and **set_upper_bounds**. To specify an unbounded dimension, you can use ±float('inf') (or ±numpy.inf) in Python to specify ±∞.

Non linear constraints of equality or inequality set using **add_inequality_constraint** and **add_equality_constraint**

In [75]:
def obj_f(x,grad):
    #set the derivatives and return the objective function value
    if grad.size > 0:
        grad[0] = 2 * (x[0] -2)
        grad[1] = 2 * (x[1] -3)
    return (x[0]-2)**2 + (x[1] -3)**2

opt = nlopt.opt(nlopt.LD_MMA, 2)
opt.set_lower_bounds([-np.inf, -np.inf])
opt.set_upper_bounds([np.inf, np.inf])
opt.set_min_objective(obj_f)
#opt.add_inequality_constraint
#opt.add_inequality_constraint
opt.set_xtol_rel(1e-6)
x = opt.optimize([1.234, 5.678])

In [76]:
minf = opt.last_optimum_value()

In [77]:
print("optimum at ", x[0], x[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

optimum at  2.0000005133944665 2.9999999431334743
minimum value =  2.6680767997385906e-13
result code =  4


Lets add a constraint that $x >= 4$ and $y <= 5$. Obviopusly mimima should come at (4,3).

In [78]:
opt = nlopt.opt(nlopt.LD_MMA, 2)
opt.set_lower_bounds([4, -np.inf])
opt.set_upper_bounds([np.inf, 5])
opt.set_min_objective(obj_f)
opt.set_xtol_rel(1e-6)
x = opt.optimize([5, 0])

In [79]:
minf = opt.last_optimum_value()

In [80]:
print("optimum at ", x[0], x[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

optimum at  4.0 2.9999993089283707
minimum value =  4.000000000000478
result code =  4


Lets add an inequality constraint constraint on plane that $y <= x - 20$ => $-x +y -20 <= 0$

Adding this we expect the point to be perpendiculas from (2,3) on line $y = x - 20$ which is ((25/2 and -15/2) by simple cordinate geometry. Let us see if we get it

In [93]:
#opt.add_inequality_constraint
def myconstraint(x, grad):
    if grad.size > 0:
        grad[0] = -1
        grad[1] = 1
    return -x[0] + x[1] +20

opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad), 1e-8)

In [94]:
x = opt.optimize([100, -100])

In [95]:
minf = opt.last_optimum_value()

In [96]:
print("optimum at ", x[0], x[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

optimum at  12.499923475929121 -7.500076520214666
minimum value =  220.49999993073078
result code =  4


Indeed we get $12.5 (= 25/2)$ and $-7.5 (= -15/2)$

For more examples refer to https://nlopt.readthedocs.io/en/latest/NLopt_Tutorial/